In [1]:
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
#from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.multioutput import MultiOutputClassifier


from sklearn.metrics import accuracy_score,confusion_matrix,classification_report,plot_confusion_matrix,roc_auc_score
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import GridSearchCV


import warnings
warnings.filterwarnings('ignore')

# Column descriptions:

# Prediction vs Norwich (23/12/2021)

In [2]:
df=pd.read_csv('Arsenal season 2021_2022 - Sheet1 (2).csv')
drop=['pred_poss','Day','Date','Round','Prediction','GF_prediction', 'GA_prediction','Predicted_formation','pred_gk_Ratings','pred_def_ratings', 'pred_mid_ratings', 'pred_forw_ratings','Opponent','Venue','Result']


opponent=pd.get_dummies(df['Opponent'],drop_first=True)
venue=pd.get_dummies(df['Venue'],drop_first=True)
result=pd.get_dummies(df['Result'],drop_first=True)

df=pd.concat([df,opponent,venue,result],axis=1)
df=df.iloc[:18]

df=df.drop(drop,axis=1)

df['Formation'].replace('3-4-2003','3-4-3',inplace=True)
df['Formation']=df['Formation'].apply(lambda x:int(x.replace('-','')))

In [5]:
train=df.iloc[:14]
test=df.iloc[14:]

In [6]:
x_train=train.drop(['GF','GA','L','W'],axis=1)
y_train=train[['L','W']]

In [7]:
x_test=test.drop(['GF','GA','L','W'],axis=1)
y_test=test[['L','W']]

In [16]:
y_train_scoring=train[['GF','GA']]
y_test_scoring=test[['GF','GA']]

    These are the inputs that are supposed to be given to the model. Each value of norwich_input represent the respective value

In [24]:
x_train.columns

Index(['xG', 'xGA', 'Poss', 'Formation', 'act_gk_ratings', 'act_def_ratings',
       'act_mid_ratings', 'act_forw_ratings', 'Brentford', 'Brighton',
       'Burnley', 'Chelsea', 'Crystal Palace', 'Everton', 'Leeds United',
       'Leicester City', 'Liverpool', 'Manchester City', 'Manchester Utd',
       'Newcastle Utd', 'Norwich City', 'Southampton', 'Tottenham', 'Watford',
       'West Ham', 'Wolves', 'Home'],
      dtype='object')

Predicting match results

In [21]:
norwich_input=[[2.033333333,0.9333333333,58,4411,7.051666667,6.963333333,7.070416667,7.343333333,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0]]

In [22]:
model = LogisticRegression(C=100)
lr=MultiOutputClassifier(model)
lr.fit(x_train,y_train)
lr.predict(norwich_input)

array([[0, 1]], dtype=uint8)

Predicting match scoring

In [20]:
model = LogisticRegression(C=100)
lr=MultiOutputClassifier(model)
lr.fit(x_train,y_train_scoring)
lr.predict(norwich_input)

array([[1., 0.]])

###    The model predicts Arsenal will win against Norwich with a score of 1-0.


*******************************************************




# Prediction vs Manchester City (1/1/2021)

    - Dropping all predicted data that were used since the model we will train is based on the actual data only.
    
    - Predicted data will only be used as an input for predicting the match results.

In [51]:
df=pd.read_csv('Arsenal season 2021_2022 - Sheet1 (3).csv')
drop=['result_prediction','predicted_poss','Day','Date','Round','GF_prediction', 'GA_prediction','Predicted_formation','pred_gk_Ratings','pred_def_ratings', 'pred_mid_ratings', 'pred_forw_ratings','Opponent','Venue','Result','GF_prediction','GA_prediction','predicted_xg','predicted_xga']


opponent=pd.get_dummies(df['Opponent'],drop_first=True)
venue=pd.get_dummies(df['Venue'],drop_first=True)
result=pd.get_dummies(df['Result'],drop_first=True)

df=pd.concat([df,opponent,venue,result],axis=1)
#needs to be change every week since there will be new match data
df=df.iloc[:19]

df=df.drop(drop,axis=1)

df['Formation'].replace('3-4-2003','3-4-3',inplace=True)
df['Formation']=df['Formation'].apply(lambda x:int(x.replace('-','')))

##### To predict match results

In [94]:
x_train=df.drop(['GF','GA','L','W'],axis=1)
y_train=df[['L','W']]

##### To predict score results

In [101]:
y_train_scoring=df[['GF','GA']]

##### Making match result predictions

In [118]:
x_train.columns

Index(['xG', 'xGA', 'Poss', 'Formation', 'act_gk_ratings', 'act_def_ratings',
       'act_mid_ratings', 'act_forw_ratings', 'Brentford', 'Brighton',
       'Burnley', 'Chelsea', 'Crystal Palace', 'Everton', 'Leeds United',
       'Leicester City', 'Liverpool', 'Manchester City', 'Manchester Utd',
       'Newcastle Utd', 'Norwich City', 'Southampton', 'Tottenham', 'Watford',
       'West Ham', 'Wolves', 'Home'],
      dtype='object')

> These are **predicted data** used as **inputs** for the model to predict(taken from excel).

In [81]:
mc_input=[[2.116666667,0.9166666667,57.83333333,4411,7.011666667,6.892916667,7.219166667,7.566666667,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1]]

In [106]:
model = LogisticRegression(C=100)
lr=MultiOutputClassifier(model)
lr.fit(x_train,y_train)
#make prediction using input for this match

print('LOSE------WIN')
lr.predict(mc_input)
#lr.predict(mc_input)

LOSE------WIN


array([[0, 1]], dtype=uint8)

##### Predicting score results

In [107]:
model = LogisticRegression(C=100)
lr=MultiOutputClassifier(model)
lr.fit(x_train,y_train_scoring)

print('ARS----MC')
lr.predict(mc_input)

ARS----MC


array([[2., 0.]])

## The model predicts Arsenal will win against Manchester City with a score of 2-0.

********************************************

# Prediction vs Burnley (23/1/2022)

In [72]:
df=pd.read_csv('Arsenal season 2021_2022 - Sheet1.csv')

Dropping the 2 match that was delayed

In [73]:
df.drop([19,21], axis=0, inplace=True)

In [74]:
#df=pd.read_csv('Arsenal season 2021_2022 - Sheet1.csv')
drop=['result_prediction','predicted_poss','Day','Date','Round','GF_prediction', 'GA_prediction','Predicted_formation','pred_gk_Ratings','pred_def_ratings', 'pred_mid_ratings', 'pred_forw_ratings','Opponent','Venue','Result','GF_prediction','GA_prediction','predicted_xg','predicted_xga']


opponent=pd.get_dummies(df['Opponent'],drop_first=True)
venue=pd.get_dummies(df['Venue'],drop_first=True)
result=pd.get_dummies(df['Result'],drop_first=True)

df=pd.concat([df,opponent,venue,result],axis=1)
#needs to be change every week since there will be new match data
df=df.iloc[:20]



df=df.drop(drop,axis=1)

df['Formation'].replace('3-4-2003','3-4-3',inplace=True)
df['Formation']=df['Formation'].apply(lambda x:int(x.replace('-','')))

In [83]:
df

,xG,xGA,Poss,Formation,act_gk_ratings,act_def_ratings,act_mid_ratings,act_forw_ratings,GF,GA,...,Newcastle Utd,Norwich City,Southampton,Tottenham,Watford,West Ham,Wolves,Home,L,W
0,1.4,1.3,66.0,4231,5.78,6.330000,6.660000,5.956667,0.0,2.0,...,0,0,0,0,0,0,0,0,1,0
1,0.3,2.9,35.0,4231,6.74,6.175000,6.280000,6.243333,0.0,2.0,...,0,0,0,0,0,0,0,1,1,0
2,0.1,3.8,20.0,343,6.29,5.570000,5.630000,6.035000,0.0,5.0,...,0,0,0,0,0,0,0,0,1,0
3,2.8,0.6,52.0,4231,6.72,6.882000,6.804000,7.570000,1.0,0.0,...,0,1,0,0,0,0,0,1,0,1
4,1.2,1.3,54.0,4141,7.57,7.028000,6.566000,6.730000,1.0,0.0,...,0,0,0,0,0,0,0,0,0,1
5,0.8,1.0,46.0,4231,6.89,6.702000,7.168333,8.470000,3.0,1.0,...,0,0,0,1,0,0,0,1,0,1
6,0.5,1.4,41.0,4231,6.66,6.725000,6.678000,5.970000,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
7,1.4,0.9,55.0,4141,6.47,6.417500,5.990000,7.046667,2.0,2.0,...,0,0,0,0,0,0,0,1,0,0
8,2.7,1.4,54.0,4231,6.49,6.757500,7.344000,6.816667,3.0,1.0,...,0,0,0,0,0,0,0,1,0,1
9,0.9,1.2,36.0,4411,8.64,7.060000,7.232500,6.805000,2.0,0.0,...,0,0,0,0,0,0,0,0,0,1


In [76]:
x_train=df.drop(['GF','GA','L','W'],axis=1)
y_train=df[['L','W']]

In [77]:
y_train_scoring=df[['GF','GA']]

In [78]:
x_train.columns

Index(['xG', 'xGA', 'Poss', 'Formation', 'act_gk_ratings', 'act_def_ratings',
       'act_mid_ratings', 'act_forw_ratings', 'Brentford', 'Brighton',
       'Burnley', 'Chelsea', 'Crystal Palace', 'Everton', 'Leeds United',
       'Leicester City', 'Liverpool', 'Manchester City', 'Manchester Utd',
       'Newcastle Utd', 'Norwich City', 'Southampton', 'Tottenham', 'Watford',
       'West Ham', 'Wolves', 'Home'],
      dtype='object')

In [79]:
burnley_input=[[2.375,0.95,49.5,4141,6.816666667,6.89625,7.15375,7.568333333,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1]]

#### Predicting match results

In [80]:
model = LogisticRegression(C=100)
lr=MultiOutputClassifier(model)
lr.fit(x_train,y_train)
#make prediction using input for this match

print('LOSE------WIN')
lr.predict(burnley_input)
#lr.predict(mc_input)

LOSE------WIN


array([[0, 1]], dtype=uint8)

#### Predicting score results

In [82]:
odel = LogisticRegression(C=100)
lr=MultiOutputClassifier(model)
lr.fit(x_train,y_train_scoring)

print('ARS----BURNLEY')
lr.predict(burnley_input)

ARS----BURNLEY


array([[1., 0.]])

# The model predicts Arsenal will win against Burnley with a score of 1-0.

***************

# Prediction vs Wolves (10/2/2022)

In [29]:
df=pd.read_csv('Arsenal season 2021_2022 - Sheet1.csv')

In [30]:
#df=pd.read_csv('Arsenal season 2021_2022 - Sheet1.csv')
drop=['result_prediction','predicted_poss','Day','Date','Round','GF_prediction', 'GA_prediction','Predicted_formation','pred_gk_Ratings','pred_def_ratings', 'pred_mid_ratings', 'pred_forw_ratings','Opponent','Venue','Result','GF_prediction','GA_prediction','predicted_xg','predicted_xga']


opponent=pd.get_dummies(df['Opponent'],drop_first=True)
venue=pd.get_dummies(df['Venue'],drop_first=True)
result=pd.get_dummies(df['Result'],drop_first=True)

df=pd.concat([df,opponent,venue,result],axis=1)
#NEEDS TO BE CHANGE EVERY WEEK SINCE THERE WILL BE NEW MATCH DATA
df=df.iloc[:21]



df=df.drop(drop,axis=1)

df['Formation'].replace('3-4-2003','3-4-3',inplace=True)
#CANNOT INCLUDE THE UPCOMING MATCH PREDICTION SINCE IT IS NAN
df['Formation']=df['Formation'].apply(lambda x:int(x.replace('-','')))

In [32]:
x_train=df.drop(['GF','GA','L','W'],axis=1)
y_train=df[['L','W']]

In [33]:
y_train_scoring=df[['GF','GA']]

Input we need to pass in

In [34]:
(x_train.columns)

Index(['xG', 'xGA', 'Poss', 'Formation', 'act_gk_ratings', 'act_def_ratings',
       'act_mid_ratings', 'act_forw_ratings', 'Brentford', 'Brighton',
       'Burnley', 'Chelsea', 'Crystal Palace', 'Everton', 'Leeds United',
       'Leicester City', 'Liverpool', 'Manchester City', 'Manchester Utd',
       'Newcastle Utd', 'Norwich City', 'Southampton', 'Tottenham', 'Watford',
       'West Ham', 'Wolves', 'Home'],
      dtype='object')

In [38]:
wolves_input=[[2.0333,0.8,55.5,4231,6.951666667,6.645625,7.159583333,7.579166667,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0]]

#### Predicting match results

In [39]:
model = LogisticRegression(C=100)
lr=MultiOutputClassifier(model)
lr.fit(x_train,y_train)
#make prediction using input for this match

print('LOSE------WIN')
lr.predict(wolves_input)

LOSE------WIN


array([[0, 1]], dtype=uint8)

#### Predicting match scores

In [42]:
model = LogisticRegression(C=100)
lr=MultiOutputClassifier(model)
lr.fit(x_train,y_train_scoring)

print('ARS----WOLVES')
lr.predict(wolves_input)

ARS----WOLVES


array([[2., 0.]])

# The model predicts Arsenal will win against Wolves with a the score of 2-0.

************

# Prediction vs Brentford (19/2/2021)

In [41]:
df=pd.read_csv('Arsenal season 2021_2022 - Sheet1.csv')

In [42]:
#df=pd.read_csv('Arsenal season 2021_2022 - Sheet1.csv')
drop=['result_prediction','predicted_poss','Day','Date','Round','GF_prediction', 'GA_prediction','Predicted_formation','pred_gk_Ratings','pred_def_ratings', 'pred_mid_ratings', 'pred_forw_ratings','Opponent','Venue','Result','GF_prediction','GA_prediction','predicted_xg','predicted_xga']


opponent=pd.get_dummies(df['Opponent'],drop_first=True)
venue=pd.get_dummies(df['Venue'],drop_first=True)
result=pd.get_dummies(df['Result'],drop_first=True)

df=pd.concat([df,opponent,venue,result],axis=1)
#NEEDS TO BE CHANGE EVERY WEEK SINCE THERE WILL BE NEW MATCH DATA
df=df.iloc[:22]



df=df.drop(drop,axis=1)

df['Formation'].replace('3-4-2003','3-4-3',inplace=True)
#CANNOT INCLUDE THE UPCOMING MATCH PREDICTION SINCE IT IS NAN
df['Formation']=df['Formation'].apply(lambda x:int(x.replace('-','')))

In [44]:
x_train=df.drop(['GF','GA','L','W'],axis=1)
y_train=df[['L','W']]

In [45]:
y_train_scoring=df[['GF','GA']]

Input that needs to be passed in

In [46]:
x_train.columns

Index(['xG', 'xGA', 'Poss', 'Formation', 'act_gk_ratings', 'act_def_ratings',
       'act_mid_ratings', 'act_forw_ratings', 'Brentford', 'Brighton',
       'Burnley', 'Chelsea', 'Crystal Palace', 'Everton', 'Leeds United',
       'Leicester City', 'Liverpool', 'Manchester City', 'Manchester Utd',
       'Newcastle Utd', 'Norwich City', 'Southampton', 'Tottenham', 'Watford',
       'West Ham', 'Wolves', 'Home'],
      dtype='object')

In [47]:
brentford_input=[[2.083333333,0.9666666667,52,4231,6.766666667,6.721916667,7.037333333,7.346666667,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1]]

#### Predicting match results

In [48]:
model = LogisticRegression(C=100)
lr=MultiOutputClassifier(model)
lr.fit(x_train,y_train)
#make prediction using input for this match

print('LOSE------WIN')
lr.predict(brentford_input)

LOSE------WIN


array([[0, 1]], dtype=uint8)

#### Predicting match scores

In [49]:
model = LogisticRegression(C=100)
lr=MultiOutputClassifier(model)
lr.fit(x_train,y_train_scoring)

print('ARS----WOLVES')
lr.predict(brentford_input)

ARS----WOLVES


array([[1., 0.]])

# The model predicts Arsenal will win against Wolves with a the score of 1-0.

***

# Prediction vs Wolves (24/2/2022)

In [3]:
df=pd.read_csv('Arsenal season 2021_2022 - Sheet1.csv')

In [5]:
#df=pd.read_csv('Arsenal season 2021_2022 - Sheet1.csv')
drop=['result_prediction','predicted_poss','Day','Date','Round','GF_prediction', 'GA_prediction','Predicted_formation','pred_gk_Ratings','pred_def_ratings', 'pred_mid_ratings', 'pred_forw_ratings','Opponent','Venue','Result','GF_prediction','GA_prediction','predicted_xg','predicted_xga']


opponent=pd.get_dummies(df['Opponent'],drop_first=True)
venue=pd.get_dummies(df['Venue'],drop_first=True)
result=pd.get_dummies(df['Result'],drop_first=True)

df=pd.concat([df,opponent,venue,result],axis=1)
#NEEDS TO BE CHANGE EVERY WEEK SINCE THERE WILL BE NEW MATCH DATA
df=df.iloc[:23]



df=df.drop(drop,axis=1)

df['Formation'].replace('3-4-2003','3-4-3',inplace=True)
#CANNOT INCLUDE THE UPCOMING MATCH PREDICTION SINCE IT IS NAN
df['Formation']=df['Formation'].apply(lambda x:int(x.replace('-','')))

In [6]:
df

,xG,xGA,Poss,Formation,act_gk_ratings,act_def_ratings,act_mid_ratings,act_forw_ratings,GF,GA,...,Newcastle Utd,Norwich City,Southampton,Tottenham,Watford,West Ham,Wolves,Home,L,W
0,1.4,1.3,66.0,4231,5.78,6.330000,6.660000,5.956667,0.0,2.0,...,0,0,0,0,0,0,0,0,1,0
1,0.3,2.9,35.0,4231,6.74,6.175000,6.280000,6.243333,0.0,2.0,...,0,0,0,0,0,0,0,1,1,0
2,0.1,3.8,20.0,343,6.29,5.570000,5.630000,6.035000,0.0,5.0,...,0,0,0,0,0,0,0,0,1,0
3,2.8,0.6,52.0,4231,6.72,6.882000,6.804000,7.570000,1.0,0.0,...,0,1,0,0,0,0,0,1,0,1
4,1.2,1.3,54.0,4141,7.57,7.028000,6.566000,6.730000,1.0,0.0,...,0,0,0,0,0,0,0,0,0,1
5,0.8,1.0,46.0,4231,6.89,6.702000,7.168333,8.470000,3.0,1.0,...,0,0,0,1,0,0,0,1,0,1
6,0.5,1.4,41.0,4231,6.66,6.725000,6.678000,5.970000,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
7,1.4,0.9,55.0,4141,6.47,6.417500,5.990000,7.046667,2.0,2.0,...,0,0,0,0,0,0,0,1,0,0
8,2.7,1.4,54.0,4231,6.49,6.757500,7.344000,6.816667,3.0,1.0,...,0,0,0,0,0,0,0,1,0,1
9,0.9,1.2,36.0,4411,8.64,7.060000,7.232500,6.805000,2.0,0.0,...,0,0,0,0,0,0,0,0,0,1


In [7]:
x_train=df.drop(['GF','GA','L','W'],axis=1)
y_train=df[['L','W']]

In [8]:
y_train_scoring=df[['GF','GA']]

In [9]:
x_train.columns

Index(['xG', 'xGA', 'Poss', 'Formation', 'act_gk_ratings', 'act_def_ratings',
       'act_mid_ratings', 'act_forw_ratings', 'Brentford', 'Brighton',
       'Burnley', 'Chelsea', 'Crystal Palace', 'Everton', 'Leeds United',
       'Leicester City', 'Liverpool', 'Manchester City', 'Manchester Utd',
       'Newcastle Utd', 'Norwich City', 'Southampton', 'Tottenham', 'Watford',
       'West Ham', 'Wolves', 'Home'],
      dtype='object')

In [10]:
wolves_input=[[1.95,0.9666666667,54,4232,6.653333333,6.859166667,7.105666667,7.12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1]]

##### Predicting match results

In [11]:
model = LogisticRegression(C=100)
lr=MultiOutputClassifier(model)
lr.fit(x_train,y_train)
#make prediction using input for this match

print('LOSE------WIN')
lr.predict(wolves_input)

LOSE------WIN


array([[0, 1]], dtype=uint8)

##### Predicting match scores

In [12]:
model = LogisticRegression(C=100)
lr=MultiOutputClassifier(model)
lr.fit(x_train,y_train_scoring)

print('ARS----WOLVES')
lr.predict(wolves_input)

ARS----WOLVES


array([[1., 0.]])

# The model predicts Arsenal will win against Wolves with a the score of 1-0

****

# Prediction vs Watford (6/3/2022)

In [3]:
df=pd.read_csv('Arsenal season 2021_2022 - Sheet1.csv')

In [4]:
#df=pd.read_csv('Arsenal season 2021_2022 - Sheet1.csv')
drop=['result_prediction','predicted_poss','Day','Date','Round','GF_prediction', 'GA_prediction','Predicted_formation','pred_gk_Ratings','pred_def_ratings', 'pred_mid_ratings', 'pred_forw_ratings','Opponent','Venue','Result','GF_prediction','GA_prediction','predicted_xg','predicted_xga']


opponent=pd.get_dummies(df['Opponent'],drop_first=True)
venue=pd.get_dummies(df['Venue'],drop_first=True)
result=pd.get_dummies(df['Result'],drop_first=True)

df=pd.concat([df,opponent,venue,result],axis=1)
#NEEDS TO BE CHANGE EVERY WEEK SINCE THERE WILL BE NEW MATCH DATA
df=df.iloc[:24]



df=df.drop(drop,axis=1)

df['Formation'].replace('3-4-2003','3-4-3',inplace=True)
#CANNOT INCLUDE THE UPCOMING MATCH PREDICTION SINCE IT IS NAN
df['Formation']=df['Formation'].apply(lambda x:int(x.replace('-','')))

In [5]:
df

,xG,xGA,Poss,Formation,act_gk_ratings,act_def_ratings,act_mid_ratings,act_forw_ratings,GF,GA,...,Newcastle Utd,Norwich City,Southampton,Tottenham,Watford,West Ham,Wolves,Home,L,W
0,1.4,1.3,66.0,4231,5.78,6.330000,6.660000,5.956667,0.0,2.0,...,0,0,0,0,0,0,0,0,1,0
1,0.3,2.9,35.0,4231,6.74,6.175000,6.280000,6.243333,0.0,2.0,...,0,0,0,0,0,0,0,1,1,0
2,0.1,3.8,20.0,343,6.29,5.570000,5.630000,6.035000,0.0,5.0,...,0,0,0,0,0,0,0,0,1,0
3,2.8,0.6,52.0,4231,6.72,6.882000,6.804000,7.570000,1.0,0.0,...,0,1,0,0,0,0,0,1,0,1
4,1.2,1.3,54.0,4141,7.57,7.028000,6.566000,6.730000,1.0,0.0,...,0,0,0,0,0,0,0,0,0,1
5,0.8,1.0,46.0,4231,6.89,6.702000,7.168333,8.470000,3.0,1.0,...,0,0,0,1,0,0,0,1,0,1
6,0.5,1.4,41.0,4231,6.66,6.725000,6.678000,5.970000,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
7,1.4,0.9,55.0,4141,6.47,6.417500,5.990000,7.046667,2.0,2.0,...,0,0,0,0,0,0,0,1,0,0
8,2.7,1.4,54.0,4231,6.49,6.757500,7.344000,6.816667,3.0,1.0,...,0,0,0,0,0,0,0,1,0,1
9,0.9,1.2,36.0,4411,8.64,7.060000,7.232500,6.805000,2.0,0.0,...,0,0,0,0,0,0,0,0,0,1


In [6]:
x_train=df.drop(['GF','GA','L','W'],axis=1)
y_train=df[['L','W']]

In [7]:
y_train_scoring=df[['GF','GA']]

In [8]:
x_train.columns

Index(['xG', 'xGA', 'Poss', 'Formation', 'act_gk_ratings', 'act_def_ratings',
       'act_mid_ratings', 'act_forw_ratings', 'Brentford', 'Brighton',
       'Burnley', 'Chelsea', 'Crystal Palace', 'Everton', 'Leeds United',
       'Leicester City', 'Liverpool', 'Manchester City', 'Manchester Utd',
       'Newcastle Utd', 'Norwich City', 'Southampton', 'Tottenham', 'Watford',
       'West Ham', 'Wolves', 'Home'],
      dtype='object')

In [9]:
watford_input=[[1.866666667,0.8166666667,55,4231,6.618333333,6.65,7.074666667,7.148333333,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0]]

#### Predicting match results

In [10]:
model = LogisticRegression(C=100)
lr=MultiOutputClassifier(model)
lr.fit(x_train,y_train)
#make prediction using input for this match

print('LOSE------WIN')
lr.predict(watford_input)

LOSE------WIN


array([[0, 1]], dtype=uint8)

#### Predicting match scores

In [11]:
model = LogisticRegression(C=100)
lr=MultiOutputClassifier(model)
lr.fit(x_train,y_train_scoring)

print('ARS----WOLVES')
lr.predict(watford_input)

ARS----WOLVES


array([[2., 0.]])

# The model predicts Arsenal will win against Watford with a the score of 2-0

******

In [6]:
df=pd.read_csv('Arsenal season 2021_2022 - Sheet1.csv')

In [7]:
drop=['result_prediction','predicted_poss','Day','Date','Round','GF_prediction', 'GA_prediction','Predicted_formation','pred_gk_Ratings','pred_def_ratings', 'pred_mid_ratings', 'pred_forw_ratings','Opponent','Venue','Result','GF_prediction','GA_prediction','predicted_xg','predicted_xga']


opponent=pd.get_dummies(df['Opponent'],drop_first=True)
venue=pd.get_dummies(df['Venue'],drop_first=True)
result=pd.get_dummies(df['Result'],drop_first=True)

df=pd.concat([df,opponent,venue,result],axis=1)
#NEEDS TO BE CHANGE EVERY WEEK SINCE THERE WILL BE NEW MATCH DATA
df=df.iloc[:25]



df=df.drop(drop,axis=1)

df['Formation'].replace('3-4-2003','3-4-3',inplace=True)
#CANNOT INCLUDE THE UPCOMING MATCH PREDICTION SINCE IT IS NAN
df['Formation']=df['Formation'].apply(lambda x:int(x.replace('-','')))

In [9]:
x_train=df.drop(['GF','GA','L','W'],axis=1)
y_train=df[['L','W']]

In [10]:
y_train_scoring=df[['GF','GA']]

In [11]:
x_train.columns

Index(['xG', 'xGA', 'Poss', 'Formation', 'act_gk_ratings', 'act_def_ratings',
       'act_mid_ratings', 'act_forw_ratings', 'Brentford', 'Brighton',
       'Burnley', 'Chelsea', 'Crystal Palace', 'Everton', 'Leeds United',
       'Leicester City', 'Liverpool', 'Manchester City', 'Manchester Utd',
       'Newcastle Utd', 'Norwich City', 'Southampton', 'Tottenham', 'Watford',
       'West Ham', 'Wolves', 'Home'],
      dtype='object')

In [14]:
leicester_input=[[1.683333333,1.05,53.66666667,4231,6.463333333,6.605,7.070833333,6.90250,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1]]

In [15]:
model = LogisticRegression(C=100)
lr=MultiOutputClassifier(model)
lr.fit(x_train,y_train)
#make prediction using input for this match

print('LOSE------WIN')
lr.predict(leicester_input)

LOSE------WIN


array([[0, 1]], dtype=uint8)

In [17]:
model = LogisticRegression(C=100)
lr=MultiOutputClassifier(model)
lr.fit(x_train,y_train_scoring)

print('ARS----Lei')
lr.predict(leicester_input)

ARS----Lei


array([[2., 0.]])

***

# Prediction vs Crystal Palace (4/4/2022)

In [23]:
df=pd.read_csv('Arsenal season 2021_2022 - Sheet1.csv')

In [24]:
drop=['result_prediction','predicted_poss','Day','Date','Round','GF_prediction', 'GA_prediction','Predicted_formation','pred_gk_Ratings','pred_def_ratings', 'pred_mid_ratings', 'pred_forw_ratings','Opponent','Venue','Result','GF_prediction','GA_prediction','predicted_xg','predicted_xga']


opponent=pd.get_dummies(df['Opponent'],drop_first=True)
venue=pd.get_dummies(df['Venue'],drop_first=True)
result=pd.get_dummies(df['Result'],drop_first=True)

df=pd.concat([df,opponent,venue,result],axis=1)
#NEEDS TO BE CHANGE EVERY WEEK SINCE THERE WILL BE NEW MATCH DATA
df=df.iloc[:28]



df=df.drop(drop,axis=1)

df['Formation'].replace('3-4-2003','3-4-3',inplace=True)
#CANNOT INCLUDE THE UPCOMING MATCH PREDICTION SINCE IT IS NAN
df['Formation']=df['Formation'].apply(lambda x:int(x.replace('-','')))

In [10]:
x_train=df.drop(['GF','GA','L','W'],axis=1)
y_train=df[['L','W']]

In [11]:
y_train_scoring=df[['GF','GA']]

In [12]:
x_train.columns

Index(['xG', 'xGA', 'Poss', 'Formation', 'act_gk_ratings', 'act_def_ratings',
       'act_mid_ratings', 'act_forw_ratings', 'Brentford', 'Brighton',
       'Burnley', 'Chelsea', 'Crystal Palace', 'Everton', 'Leeds United',
       'Leicester City', 'Liverpool', 'Manchester City', 'Manchester Utd',
       'Newcastle Utd', 'Norwich City', 'Southampton', 'Tottenham', 'Watford',
       'West Ham', 'Wolves', 'Home'],
      dtype='object')

In [14]:
palace_input=[[1.716666667,0.75,54.66666667,4231,6.415,6.632083333,7.1725,6.8675,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0]]

#### Predicting match result

In [15]:
model = LogisticRegression(C=100)
lr=MultiOutputClassifier(model)
lr.fit(x_train,y_train)
#make prediction using input for this match

print('LOSE------WIN')
lr.predict(palace_input)

LOSE------WIN


array([[0, 1]], dtype=uint8)

#### Predicting match scores

In [16]:
model = LogisticRegression(C=100)
lr=MultiOutputClassifier(model)
lr.fit(x_train,y_train_scoring)

print('ARS----Lei')
lr.predict(palace_input)

ARS----Lei


array([[2., 0.]])

# The model predict Arsenal will win against Crystal Palace with a score of 2-0

****

# Prediction vs Brighton (9/4/2022)

In [13]:
df=pd.read_csv('Arsenal season 2021_2022 - Sheet1.csv')

In [14]:
drop=['result_prediction','predicted_poss','Day','Date','Round','GF_prediction', 'GA_prediction','Predicted_formation','pred_gk_Ratings','pred_def_ratings', 'pred_mid_ratings', 'pred_forw_ratings','Opponent','Venue','Result','GF_prediction','GA_prediction','predicted_xg','predicted_xga']


opponent=pd.get_dummies(df['Opponent'],drop_first=True)
venue=pd.get_dummies(df['Venue'],drop_first=True)
result=pd.get_dummies(df['Result'],drop_first=True)

df=pd.concat([df,opponent,venue,result],axis=1)
#NEEDS TO BE CHANGE EVERY WEEK SINCE THERE WILL BE NEW MATCH DATA
df=df.iloc[:29]



df=df.drop(drop,axis=1)

df['Formation'].replace('3-4-2003','3-4-3',inplace=True)
#CANNOT INCLUDE THE UPCOMING MATCH PREDICTION SINCE IT IS NAN
df['Formation']=df['Formation'].apply(lambda x:int(x.replace('-','')))

In [16]:
x_train=df.drop(['GF','GA','L','W'],axis=1)
y_train=df[['L','W']]

In [17]:
y_train_scoring=df[['GF','GA']]

In [18]:
x_train.columns

Index(['xG', 'xGA', 'Poss', 'Formation', 'act_gk_ratings', 'act_def_ratings',
       'act_mid_ratings', 'act_forw_ratings', 'Brentford', 'Brighton',
       'Burnley', 'Chelsea', 'Crystal Palace', 'Everton', 'Leeds United',
       'Leicester City', 'Liverpool', 'Manchester City', 'Manchester Utd',
       'Newcastle Utd', 'Norwich City', 'Southampton', 'Tottenham', 'Watford',
       'West Ham', 'Wolves', 'Home'],
      dtype='object')

In [20]:
brighton_input=[[1.516666667,0.9166666667,54.66666667,4231,6.301666667,6.445,6.7875,6.768333333,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1]]

#### Predicting match result

In [22]:
model = LogisticRegression(C=100)
lr=MultiOutputClassifier(model)
lr.fit(x_train,y_train)
#make prediction using input for this match

print('LOSE------WIN')
lr.predict(brighton_input)

LOSE------WIN


array([[0, 1]], dtype=uint8)

#### Predicting match scores

In [24]:
model = LogisticRegression(C=100)
lr=MultiOutputClassifier(model)
lr.fit(x_train,y_train_scoring)

print('ARS----Bri')
lr.predict(brighton_input)

ARS----Bri


array([[2., 0.]])

# The model predict Arsenal will win against Brighton with a score of 2-0

***

# Prediction vs Southampton (16/4/2022)

In [6]:
df=pd.read_csv('Arsenal season 2021_2022 - Sheet1.csv')

In [7]:
drop=['result_prediction','predicted_poss','Day','Date','Round','GF_prediction', 'GA_prediction','Predicted_formation','pred_gk_Ratings','pred_def_ratings', 'pred_mid_ratings', 'pred_forw_ratings','Opponent','Venue','Result','GF_prediction','GA_prediction','predicted_xg','predicted_xga']


opponent=pd.get_dummies(df['Opponent'],drop_first=True)
venue=pd.get_dummies(df['Venue'],drop_first=True)
result=pd.get_dummies(df['Result'],drop_first=True)

df=pd.concat([df,opponent,venue,result],axis=1)
#NEEDS TO BE CHANGE EVERY WEEK SINCE THERE WILL BE NEW MATCH DATA
df=df.iloc[:30]



df=df.drop(drop,axis=1)

df['Formation'].replace('3-4-2003','3-4-3',inplace=True)
#CANNOT INCLUDE THE UPCOMING MATCH PREDICTION SINCE IT IS NAN
df['Formation']=df['Formation'].apply(lambda x:int(x.replace('-','')))

In [9]:
x_train=df.drop(['GF','GA','L','W'],axis=1)
y_train=df[['L','W']]

In [10]:
y_train_scoring=df[['GF','GA']]

In [11]:
x_train.columns

Index(['xG', 'xGA', 'Poss', 'Formation', 'act_gk_ratings', 'act_def_ratings',
       'act_mid_ratings', 'act_forw_ratings', 'Brentford', 'Brighton',
       'Burnley', 'Chelsea', 'Crystal Palace', 'Everton', 'Leeds United',
       'Leicester City', 'Liverpool', 'Manchester City', 'Manchester Utd',
       'Newcastle Utd', 'Norwich City', 'Southampton', 'Tottenham', 'Watford',
       'West Ham', 'Wolves', 'Home'],
      dtype='object')

In [25]:
southampton_input=[[1.383333333,0.8166666667,56,433,6.221666667,6.620555556,6.685666667,6.499166667,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0]]

#### Predicting match result

In [26]:
model = LogisticRegression(C=100)
lr=MultiOutputClassifier(model)
lr.fit(x_train,y_train)
#make prediction using input for this match

print('LOSE------WIN')
lr.predict(southampton_input)

LOSE------WIN


array([[0, 1]], dtype=uint8)

#### Predicting match scores¶

In [27]:
model = LogisticRegression(C=100)
lr=MultiOutputClassifier(model)
lr.fit(x_train,y_train_scoring)

print('ARS----South')
lr.predict(southampton_input)

ARS----South


array([[5., 5.]])

# The model predict Arsenal will win against Southampton but predict the score to tie with 5-5.

***

In [8]:
df=pd.read_csv('Arsenal season 2021_2022 - Sheet1.csv')

In [9]:
drop=['result_prediction','predicted_poss','Day','Date','Round','GF_prediction', 'GA_prediction','Predicted_formation','pred_gk_Ratings','pred_def_ratings', 'pred_mid_ratings', 'pred_forw_ratings','Opponent','Venue','Result','GF_prediction','GA_prediction','predicted_xg','predicted_xga']


opponent=pd.get_dummies(df['Opponent'],drop_first=True)
venue=pd.get_dummies(df['Venue'],drop_first=True)
result=pd.get_dummies(df['Result'],drop_first=True)

df=pd.concat([df,opponent,venue,result],axis=1)
#NEEDS TO BE CHANGE EVERY WEEK SINCE THERE WILL BE NEW MATCH DATA
df=df.iloc[:31]



df=df.drop(drop,axis=1)

df['Formation'].replace('3-4-2003','3-4-3',inplace=True)
df['Formation'].replace('4-3-2003','4-3-3',inplace=True)

#CANNOT INCLUDE THE UPCOMING MATCH PREDICTION SINCE IT IS NAN
df['Formation']=df['Formation'].apply(lambda x:int(x.replace('-','')))

In [11]:
x_train=df.drop(['GF','GA','L','W'],axis=1)
y_train=df[['L','W']]

In [12]:
y_train_scoring=df[['GF','GA']]

In [13]:
x_train.columns


Index(['xG', 'xGA', 'Poss', 'Formation', 'act_gk_ratings', 'act_def_ratings',
       'act_mid_ratings', 'act_forw_ratings', 'Brentford', 'Brighton',
       'Burnley', 'Chelsea', 'Crystal Palace', 'Everton', 'Leeds United',
       'Leicester City', 'Liverpool', 'Manchester City', 'Manchester Utd',
       'Newcastle Utd', 'Norwich City', 'Southampton', 'Tottenham', 'Watford',
       'West Ham', 'Wolves', 'Home'],
      dtype='object')

In [14]:
chelsea_input=[[1.45,0.7,59.16666667,3421,6.3,6.627083333,6.593333333,6.825,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]]

#### Predicting match result

In [15]:
model = LogisticRegression(C=100)
lr=MultiOutputClassifier(model)
lr.fit(x_train,y_train)
#make prediction using input for this match

print('LOSE------WIN')
lr.predict(chelsea_input)

LOSE------WIN


array([[0, 1]], dtype=uint8)

#### Predicting match scores

In [17]:
model = LogisticRegression(C=100)
lr=MultiOutputClassifier(model)
lr.fit(x_train,y_train_scoring)

print('ARS----Chel')
lr.predict(chelsea_input)

ARS----Chel


array([[0., 1.]])

# The model predict Arsenal will win against Chelsea but predict the score to be 1-0 for Chelsea.

***